In [5]:
!pip install GPUtil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=ecaa7e09b01c069f079504bdb40af4a84bae6c33653670592af0c075dfb8d4ff
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_pretrained_bert googletrans

In [10]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 4.1 MB/s eta 0:00:00


In [11]:
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from preprocessing import preprocess


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load the MARBERT tokenizer
MARBERT_tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

# Load the MARBERT model for multi-label classification
MARBERT_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=11)

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
dataset_train_raw = preprocess("2018-E-c-Ar-train.xlsx")

In [16]:
# Preprocess your dataset
texts_train = dataset_train_raw["Tweet"].tolist()

encoded_inputs_train = MARBERT_tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt")
labels_train = dataset_train_raw.iloc[:, 2:].values  # Labels start from the third column

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
# Convert the labels to tensors
labels_train = torch.tensor(labels_train, dtype=torch.float)

# Create a TensorDataset from the encoded inputs and labels
dataset_train = TensorDataset(encoded_inputs_train.input_ids, encoded_inputs_train.attention_mask, labels_train)


In [18]:
print("Length of dataset_train:", len(dataset_train))
print("Shape of input_ids tensor in dataset_train:", dataset_train.tensors[0].shape)
print("Shape of attention_mask tensor in dataset_train:", dataset_train.tensors[1].shape)
print("Shape of labels tensor in dataset_train:", dataset_train.tensors[2].shape)

Length of dataset_train: 2278
Shape of input_ids tensor in dataset_train: torch.Size([2278, 41])
Shape of attention_mask tensor in dataset_train: torch.Size([2278, 41])
Shape of labels tensor in dataset_train: torch.Size([2278, 11])


In [36]:
# Define hyperparameters and optimizer
batch_size = 16
num_epochs = 4
learning_rate = 1e-4
optimizer = AdamW(MARBERT_model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
# Create a DataLoader for the training dataset
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [38]:
# Perform training
MARBERT_model.train()

for epoch in range(num_epochs):
    total_loss = 0
    
    for batch in dataloader_train:
        # Retrieve the batch inputs and labels
        input_ids, attention_mask, batch_labels = batch
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update total loss
        total_loss += loss.item()
    
    # Print average loss for the epoch
    average_loss = total_loss / len(dataloader_train)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")


Epoch 1/4 - Average Loss: 0.1694
Epoch 2/4 - Average Loss: 0.1812
Epoch 3/4 - Average Loss: 0.1379
Epoch 4/4 - Average Loss: 0.0915


In [43]:
model_save_name = 'marbert_4epochs.pt'

In [44]:
# Saving the trained model
torch.save(MARBERT_model.state_dict(), model_save_name)

In [ ]:
files.download('checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load saved model
MARBERT_model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [35]:
dataset_test_raw = preprocess("2018-E-c-Ar-test-gold.xlsx")

In [28]:
# Preprocess your dataset
texts_test = dataset_test_raw["Tweet"].tolist()

encoded_inputs_test = MARBERT_tokenizer(texts_test, padding=True, truncation=True, return_tensors="pt")
labels_test = dataset_test_raw.iloc[:, 2:].values  # Assuming labels start from the third column

In [29]:
# Convert the labels to tensors
labels_test = torch.tensor(labels_test, dtype=torch.float)

# Create a TensorDataset from the encoded inputs and labels
dataset_test = TensorDataset(encoded_inputs_test.input_ids, encoded_inputs_test.attention_mask, labels_test)

In [31]:
# Create a DataLoader for the test dataset

dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Lists to store predicted and true labels
all_predictions = []
all_true_labels = []

# Disable gradient calculation
with torch.no_grad():
    # Set the model to evaluation mode
    MARBERT_model.eval()

    # Iterate over the test dataloader
    for batch in dataloader_test:
        # Retrieve the batch inputs and labels
        input_ids, attention_mask, batch_labels = batch

        # Move the batch tensors to the device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        batch_labels = batch_labels.to(device)

        # Forward pass
        outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to probabilities
        probabilities = torch.sigmoid(logits)

        # Convert probabilities to predicted labels (0 or 1)
        predictions = torch.round(probabilities)

        # Append predictions and true labels to the lists
        all_predictions.extend(predictions.cpu().numpy())
        all_true_labels.extend(batch_labels.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(all_true_labels, all_predictions)
precision = precision_score(all_true_labels, all_predictions, average='weighted')
recall = recall_score(all_true_labels, all_predictions, average='weighted')
f1 = f1_score(all_true_labels, all_predictions, average='weighted')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Accuracy: 0.2688
Precision: 0.7205
Recall: 0.6513
F1-Score: 0.6712


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Looping on all test sentences and showing the output probabilities.
# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the output file path
output_file = "predicted_probabilities.txt"

test_sentences = dataset_test_raw["Tweet"].tolist()

# Open the output file in write mode
with open(output_file, "w") as file:
    for sentence in test_sentences:
        file.write("******************************************************************\n")
        file.write(f"________________{sentence}____________________\n")

        # Tokenize the sentence
        encoded_input = MARBERT_tokenizer.encode_plus(
            sentence,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Move the input tensors to the device
        input_ids = encoded_input["input_ids"].to(device)
        attention_mask = encoded_input["attention_mask"].to(device)

        # Set the model to evaluation mode
        MARBERT_model.eval()

        # Move the model to the device
        MARBERT_model.to(device)

        # Disable gradient calculation
        with torch.no_grad():
            # Forward pass
            outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Convert logits to probabilities
        probabilities = torch.sigmoid(logits)

        label_names = ["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust"]  

        # Write the predicted probabilities for each label to the file
        for i, label_name in enumerate(label_names):
            file.write(f"{label_name}: {probabilities[0][i].item():.4f}\n")
            
